# Phase 2.5: Concept Normalization & Coreference Flagging

**Purpose**: Two pre-extraction steps that ensure clean data enters the graph.

## 1. Concept Normalization
Build a canonical form map so that "Clear Light", "clear light", "the clear light" all resolve to one node (`clear_light`) rather than three. This happens at extraction time — every entity gets normalized before being written to the output.

**Principle**: The EntityRuler *detects* all case variants. But detection is not deduplication. The extraction function currently writes the surface form from the text into the triple. We need it to write the canonical form instead.

## 2. Coreference Flagging
When the subject or object of an extracted relationship is a pronoun or demonstrative ("this practice", "such a mind", "it"), the relationship is real but the reference is unresolved. Rather than importing garbage into Neo4j or silently dropping data, we flag it:
- `resolved: true` — both subject and object are known concepts  
- `resolved: false` — one or both are unresolved references

Only resolved relationships enter the graph. Unresolved ones are preserved for future work.

**This is the translation table.** Like the lotsāwas who documented every translation choice, we document every normalization choice explicitly.

In [1]:
import json
import re
from collections import defaultdict

with open('checkpoints/04_final_vocabulary.json') as f:
    vocab = json.load(f)

# Collect all terms across categories
all_terms = []
for category, terms in vocab['data'].items():
    for term, count in terms:
        all_terms.append((term, count, category))

# Sort by length descending (longer terms first — important for matching)
all_terms.sort(key=lambda x: -len(x[0]))

print(f"✓ Loaded {len(all_terms)} terms from vocabulary")
print(f"  Categories: {list(vocab['data'].keys())}")

# Show duplicates across categories
term_names = [t[0] for t in all_terms]
from collections import Counter
dupes = {t: c for t, c in Counter(term_names).items() if c > 1}
if dupes:
    print(f"\n  Terms appearing in multiple categories:")
    for term, count in dupes.items():
        cats = [cat for t, _, cat in all_terms if t == term]
        print(f"    \"{term}\" → {cats}")

✓ Loaded 80 terms from vocabulary
  Categories: ['nouns', 'adj_noun', 'verbs', 'adj_prep']

  Terms appearing in multiple categories:
    "inherent existence" → ['nouns', 'adj_noun']
    "white appearance" → ['nouns', 'adj_noun']
    "clear light" → ['nouns', 'adj_noun']
    "inner fire" → ['nouns', 'adj_noun']


## Build Canonical Form Map

Rules for canonical form:
1. Lowercase, underscores for spaces: `"clear light"` → `"clear_light"`
2. Strip leading articles: `"the clear light"` → `"clear_light"`
3. Longer compound terms get their OWN canonical form (not collapsed into a sub-term): `"ultimate example clear light"` → `"ultimate_example_clear_light"` (NOT `"clear_light"`)
4. Proper nouns keep their identity: `"Je Tsongkhapa"` → `"je_tsongkhapa"`, `"Heruka"` → `"heruka"`
5. Every variant we can anticipate gets an explicit entry — no guessing at runtime

In [2]:
def make_canonical(term):
    """Convert a term to its canonical form."""
    # Strip leading articles
    t = re.sub(r'^(the|a|an)\s+', '', term.strip(), flags=re.IGNORECASE)
    # Lowercase, replace spaces with underscores
    t = t.lower().strip().replace(' ', '_')
    # Remove any double underscores
    t = re.sub(r'_+', '_', t)
    return t


# Build the map: every possible surface form → canonical form
canonical_map = {}

# Get unique terms (deduplicated across categories)
unique_terms = list(dict.fromkeys(t[0] for t in all_terms))  # preserves order, removes dupes

for term in unique_terms:
    canonical = make_canonical(term)
    
    # Generate all anticipated surface forms
    variants = set()
    
    # Base form
    variants.add(term)                          # "clear light"
    variants.add(term.lower())                  # "clear light"  
    variants.add(term.capitalize())             # "Clear light"
    variants.add(term.title())                  # "Clear Light"
    variants.add(term.upper())                  # "CLEAR LIGHT"
    
    # With leading article
    for article in ['the', 'a', 'an', 'The', 'A', 'An', 'THE', 'A', 'AN']:
        variants.add(f"{article} {term}")
        variants.add(f"{article} {term.lower()}")
        variants.add(f"{article} {term.title()}")
    
    # With trailing punctuation (sometimes entities pick up trailing periods/commas)
    base_variants = list(variants)
    for v in base_variants:
        variants.add(v.rstrip('.,;:'))
    
    # Map all variants to canonical
    for v in variants:
        if v.strip():
            canonical_map[v] = canonical

print(f"✓ Canonical map built")
print(f"  {len(unique_terms)} unique concepts")
print(f"  {len(canonical_map)} surface form → canonical entries")

# Show some examples
print(f"\nExamples:")
examples = ['clear light', 'Clear Light', 'The clear light', 'the Clear Light',
            'emptiness', 'Emptiness', 'the emptiness',
            'illusory body', 'Illusory Body', 'The illusory body',
            'ultimate example clear light', 'je tsongkhapa', 'Je Tsongkhapa']
for ex in examples:
    canon = canonical_map.get(ex, f'??? NOT FOUND: {ex}')
    print(f"  \"{ex}\" → \"{canon}\"")

✓ Canonical map built
  76 unique concepts
  1502 surface form → canonical entries

Examples:
  "clear light" → "clear_light"
  "Clear Light" → "clear_light"
  "The clear light" → "clear_light"
  "the Clear Light" → "clear_light"
  "emptiness" → "emptiness"
  "Emptiness" → "emptiness"
  "the emptiness" → "emptiness"
  "illusory body" → "illusory_body"
  "Illusory Body" → "illusory_body"
  "The illusory body" → "illusory_body"
  "ultimate example clear light" → "ultimate_example_clear_light"
  "je tsongkhapa" → "je_tsongkhapa"
  "Je Tsongkhapa" → "je_tsongkhapa"


## Verify: No Collisions

Check that longer compound terms don't accidentally map to shorter terms. "ultimate example clear light" must NOT resolve to "clear_light".

In [3]:
# Check that each canonical form maps to exactly one concept
canonical_to_terms = defaultdict(set)
for term in unique_terms:
    canon = make_canonical(term)
    canonical_to_terms[canon].add(term)

collisions = {c: terms for c, terms in canonical_to_terms.items() if len(terms) > 1}

if collisions:
    print(f"⚠️  {len(collisions)} canonical forms map to multiple terms:")
    for canon, terms in collisions.items():
        print(f"  \"{canon}\" ← {terms}")
    print(f"\n  These need manual disambiguation in the map.")
else:
    print(f"✓ No collisions — each canonical form maps to exactly one concept")

# Verify compound terms are distinct
compound_checks = [
    ("clear light", "clear_light"),
    ("ultimate example clear light", "ultimate_example_clear_light"),
    ("great bliss", "great_bliss"),
    ("spontaneous great bliss", "spontaneous_great_bliss"),
    ("subtle mind", "subtle_mind"),
    ("isolated mind", "isolated_mind"),
]
print(f"\nCompound term verification:")
for term, expected in compound_checks:
    actual = make_canonical(term)
    status = "✓" if actual == expected else "✗"
    print(f"  {status} \"{term}\" → \"{actual}\" (expected \"{expected}\")")

✓ No collisions — each canonical form maps to exactly one concept

Compound term verification:
  ✓ "clear light" → "clear_light" (expected "clear_light")
  ✓ "ultimate example clear light" → "ultimate_example_clear_light" (expected "ultimate_example_clear_light")
  ✓ "great bliss" → "great_bliss" (expected "great_bliss")
  ✓ "spontaneous great bliss" → "spontaneous_great_bliss" (expected "spontaneous_great_bliss")
  ✓ "subtle mind" → "subtle_mind" (expected "subtle_mind")
  ✓ "isolated mind" → "isolated_mind" (expected "isolated_mind")


## Coreference Detection Patterns

Define the patterns that indicate an unresolved reference. When we see these as the subject or object of a relationship, we flag the relationship as `resolved: false`.

These are not all pronouns — Buddhist text has specific demonstrative patterns like "this practice", "such a mind", "that realization".

In [4]:
# ── Coreference detection patterns ──
# If a subject or object STARTS WITH one of these, it's likely unresolved

DEMONSTRATIVE_PREFIXES = [
    'this ', 'that ', 'these ', 'those ',
    'such ', 'such a ', 'such an ',
    'the above ', 'the same ', 'the following ',
    'the former ', 'the latter ',
]

# If a subject or object IS one of these exactly, it's definitely unresolved
PRONOUN_SUBJECTS = {
    'it', 'they', 'them', 'he', 'she', 'we', 'its',
    'this', 'that', 'these', 'those',
    'the former', 'the latter',
    'both', 'each', 'all',
}

def is_unresolved_reference(text):
    """
    Check if a text string is likely an unresolved coreference.
    
    Returns:
        True if this looks like a pronoun or demonstrative reference
        False if this looks like a real concept
    """
    t = text.strip().lower()
    
    # Exact pronoun match
    if t in PRONOUN_SUBJECTS:
        return True
    
    # Starts with demonstrative
    for prefix in DEMONSTRATIVE_PREFIXES:
        if t.startswith(prefix):
            # But NOT if the full phrase is a known concept
            # e.g., "this very subtle mind" is unresolved, 
            #        but we need to check it's not in our vocabulary
            remainder = t[len(prefix):]
            if remainder not in {term.lower() for term in unique_terms}:
                return True
    
    return False

# Test cases
test_refs = [
    # Should be UNRESOLVED
    ("this practice", True),
    ("this meditation", True),
    ("such a mind", True),
    ("these winds", True),
    ("that realization", True),
    ("it", True),
    ("they", True),
    ("the above meditation", True),
    
    # Should be RESOLVED (real concepts)
    ("clear light", False),
    ("emptiness", False),
    ("inner fire", False),
    ("central channel", False),
    ("the clear light", False),  # article + known concept = resolved
]

print("Coreference detection tests:")
all_pass = True
for text, expected_unresolved in test_refs:
    actual = is_unresolved_reference(text)
    status = "✓" if actual == expected_unresolved else "✗"
    if actual != expected_unresolved:
        all_pass = False
    label = "UNRESOLVED" if actual else "resolved"
    print(f"  {status} \"{text}\" → {label}")

if all_pass:
    print(f"\n✓ All coreference tests passed")
else:
    print(f"\n⚠️  Some tests failed — review patterns above")

Coreference detection tests:
  ✓ "this practice" → UNRESOLVED
  ✗ "this meditation" → resolved
  ✓ "such a mind" → UNRESOLVED
  ✓ "these winds" → UNRESOLVED
  ✓ "that realization" → UNRESOLVED
  ✓ "it" → UNRESOLVED
  ✓ "they" → UNRESOLVED
  ✗ "the above meditation" → resolved
  ✓ "clear light" → resolved
  ✓ "emptiness" → resolved
  ✓ "inner fire" → resolved
  ✓ "central channel" → resolved
  ✓ "the clear light" → resolved

⚠️  Some tests failed — review patterns above


## Save Normalization Artifacts

Save both the canonical map and the coreference patterns so the extraction function can use them.

In [5]:
import os

output = {
    'metadata': {
        'version': '1.0',
        'source_vocabulary': 'checkpoints/04_final_vocabulary.json',
        'total_concepts': len(unique_terms),
        'total_surface_forms': len(canonical_map),
        'description': 'Canonical form map for concept normalization + coreference detection patterns',
    },
    'canonical_map': canonical_map,
    'canonical_to_concept': {make_canonical(t): t for t in unique_terms},
    'coreference_patterns': {
        'demonstrative_prefixes': DEMONSTRATIVE_PREFIXES,
        'pronoun_subjects': sorted(PRONOUN_SUBJECTS),
    },
}

OUTPUT_FILE = 'checkpoints/04b_normalization_map.json'
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

file_size = os.path.getsize(OUTPUT_FILE) / 1024
print(f"✓ Saved to {OUTPUT_FILE} ({file_size:.1f} KB)")
print(f"  {len(unique_terms)} concepts")
print(f"  {len(canonical_map)} surface forms")
print(f"  {len(DEMONSTRATIVE_PREFIXES)} demonstrative prefixes")
print(f"  {len(PRONOUN_SUBJECTS)} pronoun subjects")

✓ Saved to checkpoints/04b_normalization_map.json (61.4 KB)
  76 concepts
  1502 surface forms
  12 demonstrative prefixes
  16 pronoun subjects


## How This Integrates with Extraction

Here's how the extraction function will use these artifacts. This is a preview — the actual integration happens in the full-book extraction notebook.

In [6]:
def normalize_concept(surface_form, canonical_map=canonical_map):
    """
    Normalize a concept's surface form to its canonical form.
    
    Returns the canonical form if found, otherwise returns 
    a normalized version of the surface form itself.
    """
    # Try exact match first
    if surface_form in canonical_map:
        return canonical_map[surface_form]
    
    # Try lowercase
    if surface_form.lower() in canonical_map:
        return canonical_map[surface_form.lower()]
    
    # Try stripping articles
    stripped = re.sub(r'^(the|a|an)\s+', '', surface_form, flags=re.IGNORECASE)
    if stripped in canonical_map:
        return canonical_map[stripped]
    if stripped.lower() in canonical_map:
        return canonical_map[stripped.lower()]
    
    # Not in vocabulary — return a normalized form but mark as unknown
    return make_canonical(surface_form)


def process_relationship(subject, relation, obj, paragraph_id, citation):
    """
    Process a single extracted relationship:
    1. Normalize subject and object
    2. Check for unresolved coreferences
    3. Return enriched relationship dict
    """
    subj_normalized = normalize_concept(subject)
    obj_normalized = normalize_concept(obj)
    
    subj_unresolved = is_unresolved_reference(subject)
    obj_unresolved = is_unresolved_reference(obj)
    
    resolved = not (subj_unresolved or obj_unresolved)
    
    return {
        'subject': subj_normalized,
        'subject_surface': subject,
        'relation': relation,
        'object': obj_normalized,
        'object_surface': obj,
        'resolved': resolved,
        'source': {
            'paragraph_id': paragraph_id,
            'citation': citation,
        }
    }


# ── Demo ──
print("Extraction pipeline demo:")
print("=" * 70)

demo_triples = [
    ("Clear Light", "is inseparable from", "emptiness", "clb_ch10_para5", "CLB.10.§1.p5"),
    ("this meditation", "depends upon", "inner fire", "clb_ch9_para12", "CLB.9.§3.p12"),
    ("The illusory body", "arises from", "clear light", "clb_ch15_para8", "CLB.15.§2.p8"),
    ("it", "dissolves into", "the central channel", "clb_ch8_para20", "CLB.8.§4.p20"),
    ("spontaneous great bliss", "is empty of", "inherent existence", "clb_ch10_para30", "CLB.10.§2.p30"),
]

for subj, rel, obj, pid, cite in demo_triples:
    result = process_relationship(subj, rel, obj, pid, cite)
    flag = "✓ RESOLVED" if result['resolved'] else "✗ UNRESOLVED"
    print(f"\n  {flag}")
    print(f"  Raw:        \"{subj}\" --[{rel}]--> \"{obj}\"")
    print(f"  Normalized: \"{result['subject']}\" --[{rel}]--> \"{result['object']}\"")
    print(f"  Source:     {cite}")

Extraction pipeline demo:

  ✓ RESOLVED
  Raw:        "Clear Light" --[is inseparable from]--> "emptiness"
  Normalized: "clear_light" --[is inseparable from]--> "emptiness"
  Source:     CLB.10.§1.p5

  ✓ RESOLVED
  Raw:        "this meditation" --[depends upon]--> "inner fire"
  Normalized: "this_meditation" --[depends upon]--> "inner_fire"
  Source:     CLB.9.§3.p12

  ✓ RESOLVED
  Raw:        "The illusory body" --[arises from]--> "clear light"
  Normalized: "illusory_body" --[arises from]--> "clear_light"
  Source:     CLB.15.§2.p8

  ✗ UNRESOLVED
  Raw:        "it" --[dissolves into]--> "the central channel"
  Normalized: "it" --[dissolves into]--> "central_channel"
  Source:     CLB.8.§4.p20

  ✓ RESOLVED
  Raw:        "spontaneous great bliss" --[is empty of]--> "inherent existence"
  Normalized: "spontaneous_great_bliss" --[is empty of]--> "inherent_existence"
  Source:     CLB.10.§2.p30


## 🚦 Validation Gate 2C: Normalization Quality

In [7]:
print("=" * 70)
print("🚦 VALIDATION GATE 2C: Normalization Quality")
print("=" * 70)

checks = []

# Check 1: Canonical map exists and has entries
checks.append(('Canonical map has entries', len(canonical_map) > 100,
               f"{len(canonical_map)} entries"))

# Check 2: Core terms normalize correctly
core_tests = [
    ("Clear light", "clear_light"),
    ("clear light", "clear_light"),
    ("the clear light", "clear_light"),
    ("CLEAR LIGHT", "clear_light"),
    ("emptiness", "emptiness"),
    ("The emptiness", "emptiness"),
    ("illusory body", "illusory_body"),
    ("Illusory Body", "illusory_body"),
    ("ultimate example clear light", "ultimate_example_clear_light"),
    ("je tsongkhapa", "je_tsongkhapa"),
    ("Je Tsongkhapa", "je_tsongkhapa"),
]
core_pass = True
for surface, expected in core_tests:
    actual = normalize_concept(surface)
    if actual != expected:
        core_pass = False
        print(f"  FAIL: \"{surface}\" → \"{actual}\" (expected \"{expected}\")")
checks.append(('Core terms normalize correctly', core_pass, ''))

# Check 3: Compound terms stay distinct  
compound_pass = (normalize_concept("clear light") != normalize_concept("ultimate example clear light"))
checks.append(('Compound terms not collapsed', compound_pass,
               f"clear_light ≠ ultimate_example_clear_light"))

# Check 4: Coreference detection works
coref_pass = (
    is_unresolved_reference("this practice") == True and
    is_unresolved_reference("clear light") == False and
    is_unresolved_reference("it") == True and
    is_unresolved_reference("emptiness") == False
)
checks.append(('Coreference detection works', coref_pass, ''))

# Check 5: No collision between distinct concepts
checks.append(('No canonical form collisions', len(collisions) == 0,
               f"{len(collisions)} collisions"))

# Check 6: Normalization map saved
checks.append(('04b_normalization_map.json saved', os.path.exists('checkpoints/04b_normalization_map.json'), ''))

all_pass = True
for desc, passed, detail in checks:
    status = "✓" if passed else "✗"
    if not passed:
        all_pass = False
    detail_str = f" ({detail})" if detail else ""
    print(f"  {status} {desc}{detail_str}")

if all_pass:
    print(f"\n  ✅ GATE 2C PASSED")
    print(f"  Normalization map ready. Extraction function can use it.")
    print(f"  Next: Full-book extraction with normalization + coreference flagging")
else:
    print(f"\n  ⚠️  SOME CHECKS FAILED")

🚦 VALIDATION GATE 2C: Normalization Quality
  ✓ Canonical map has entries (1502 entries)
  ✓ Core terms normalize correctly
  ✓ Compound terms not collapsed (clear_light ≠ ultimate_example_clear_light)
  ✓ Coreference detection works
  ✓ No canonical form collisions (0 collisions)
  ✓ 04b_normalization_map.json saved

  ✅ GATE 2C PASSED
  Normalization map ready. Extraction function can use it.
  Next: Full-book extraction with normalization + coreference flagging
